# 그래프 컨볼류션

- 그래프 표현방식과 그래프 컨볼류션 모델의 이해

![Molecular Graph](https://github.com/deepchem/deepchem/blob/master/examples/tutorials/assets/basic_graphs.gif?raw=1)



## Graph Convolutions 개념

- 일반 CNN
 - 이미지 처리, 시계열 처리 등에 사용
 - 신호가 여러 컨볼류션 계층을 통과한다. 주변의 샘플들에 필터를 적용하며 어떤 추상적인 패턴을 추출한다
 - 가끔 풀링을 수행하여 패턴신호(특성)의 이동과 정보 축약을 수행한다
- 그래프 CNN
 - 일반 CNN과 유사하나 이미지 등이 아니라 그래프를 대상으로 동작한다
 - 주변 샘플 전체가 아니라 그래프로 연결된 샘플들을 사용하여 컨볼류션과 풀링을 수행한다


# import

In [1]:
!pip install --pre deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 608 kB 8.3 MB/s 
     |████████████████████████████████| 36.8 MB 36 kB/s 


In [6]:
import deepchem as dc

from deepchem.models.layers import GraphConv, GraphPool, GraphGather
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol
import tensorflow as tf
import tensorflow.keras.layers as layers

import numpy as np

# GCN 사용 방법

In [2]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

- `GraphConvModel`  wraps a standard graph convolutional architecture underneath the hood for user convenience.

In [3]:
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification')
model.fit(train_dataset, nb_epoch=50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(435,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(435, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1396,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16

0.3036912155151367

- `dc.metrics`을 이용하여 성능을 특정한다
- 평가함수 `model.evaluate()`을 사용한다

In [4]:
metric1 = dc.metrics.Metric(dc.metrics.roc_auc_score)
metric2 = dc.metrics.Metric(dc.metrics.accuracy_score)
print('Training set score:', model.evaluate(train_dataset, [metric1, metric2], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric1, metric2], transformers))

Training set score: {'roc_auc_score': 0.9648448979346483, 'accuracy_score': 0.8838069391230311}
Test set score: {'roc_auc_score': 0.6845173257758642, 'accuracy_score': 0.7670068027210885}


# 그래프 컨볼류션 상세 구현

- `GraphConvModel` 객체 사용법도 편리하고 결과 성능도 좋다

- 이번에는 GraphConvModel의 내용을 직접 구현해보겠다

 -  `GraphConv` layer: implements the graph convolution. The graph convolution combines per-node feature vectors in a nonlinear fashion with the feature vectors for neighboring nodes.  This "blends" information in local neighborhoods of a graph.

- `GraphPool` layer: does a max-pooling over the feature vectors of atoms in a neighborhood. You can think of this layer as analogous to a max-pooling layer for 2D convolutions

- `GraphGather`: Many graph convolutional networks manipulate feature vectors per graph-node. For a molecule for example, each node might represent an atom, and the network would manipulate atomic feature vectors that summarize the local chemistry of the atom.  
 - However, at the end of the application, we will likely want to work with a molecule level feature representation. 
 - This layer creates a graph level feature vector by combining all the node-level feature vectors.

- 이외에 [Dense](https://keras.io/api/layers/core_layers/dense/), [BatchNormalization](https://keras.io/api/layers/normalization_layers/batch_normalization/), [Softmax](https://keras.io/api/layers/activation_layers/softmax/) 를 사용한다

In [5]:
batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

- There are two convolutional blocks, each consisting of a `GraphConv`, followed by batch normalization, followed by a `GraphPool` to do max pooling.  

- We finish up with a dense layer, another batch normalization, a `GraphGather` to combine the data from all the different nodes, and a final dense layer to produce the global output. 

- Let's now create the DeepChem model which will be a wrapper around the Keras model that we just created.

In [7]:
model = dc.models.KerasModel(MyGraphConvModel(), loss=dc.models.losses.CategoricalCrossEntropy())

- What are the inputs to this model?  
 - A graph convolution requires a complete description of each molecule, including the list of nodes (atoms) and a description of which ones are bonded to each other.  
 - In fact, if we inspect the dataset we see that the feature array contains Python objects of type `ConvMol`.

In [8]:
test_dataset.X[0]

- Models expect arrays of numbers as their inputs, not Python objects.  We must convert the `ConvMol` objects into the particular set of arrays expected by the `GraphConv`, `GraphPool`, and `GraphGather` layers.  
- Fortunately, the `ConvMol` class includes the code to do this, as well as to combine all the molecules in a batch to create a single set of arrays.

- The following code creates a Python generator that given a batch of data generates the lists of inputs, labels, and weights whose values are Numpy arrays. 
 - `atom_features` holds a feature vector of length 75 for each atom. The other inputs are required to support minibatching in TensorFlow. 
 - `degree_slice` is an indexing convenience that makes it easy to locate atoms from all molecules with a given degree. 
 - `membership` determines the membership of atoms in molecules (atom `i` belongs to molecule `membership[i]`). `deg_adjs` is a list that contains adjacency lists grouped by atom degree. For more details, check out the [code](https://github.com/deepchem/deepchem/blob/master/deepchem/feat/mol_graphs.py).

In [9]:
def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

- train the model using `fit_generator(generator)` which will use the generator we've defined to train the model.

In [10]:
model.fit_generator(data_generator(train_dataset, epochs=50))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/my_graph_conv_model/graph_pool_3/Reshape_14:0", shape=(408,), dtype=int32), values=Tensor("gradient_tape/my_graph_conv_model/graph_pool_3/Reshape_13:0", shape=(408, 128), dtype=float32), dense_shape=Tensor("gradient_tape/my_graph_conv_model/graph_pool_3/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/my_graph_conv_model/graph_pool_3/Reshape_17:0", shape=(1460,), dtype=int32), values=Tensor("gradient_tape/my_graph_conv_model/graph_pool_3/Reshape_16:0", shape=(1460, 128), dtype=float32), dense_shape=Tensor(

0.2217037010192871

- evaluate its performance. We again have to use our defined generator to evaluate model performance.

In [11]:
print('Training set score:', model.evaluate_generator(data_generator(train_dataset), [metric1, metric2], transformers))
print('Test set score:', model.evaluate_generator(data_generator(test_dataset), [metric1, metric2], transformers))

Training set score: {'roc_auc_score': 0.840248199699094, 'accuracy_score': 0.8249735449735449}
Test set score: {'roc_auc_score': 0.6399126649361838, 'accuracy_score': 0.7332291666666667}


- The model we've constructed behaves nearly identically to `GraphConvModel`. 